###### vx - retrieve hardware execution

___________________
___________________
___________________

In [1]:
%load_ext autoreload
%autoreload 2

from troter_utils import *

### Quantum Devices


In [2]:
# load IBMQ Account data
provider = IBMQ.load_account()

# Get backend for experiment
provider = IBMQ.get_provider(hub='ibm-q-community', group='ibmquantumawards', project='open-science-22')
jakarta = provider.get_backend('ibmq_jakarta')

# Simulated backend based on ibmq_jakarta's device noise profile
sim_noisy_jakarta = QasmSimulator.from_backend(provider.get_backend('ibmq_jakarta'))

# Noiseless simulated backend
sim_no_noise = QasmSimulator()

____________

## Retrieve jobs

### By ID

In [6]:
with open('results/dict_jobs_ids_first_order_2022-04-08.pkl', 'rb') as f:
    jobs_dict_1st_order_ids = pickle.load(f)
    
with open('results/dict_jobs_ids_second_order_2022-04-06.pkl', 'rb') as f:
    jobs_dict_2nd_order_ids = pickle.load(f)

In [25]:
results_hardware_1st_order = {'order' : [], 
                              'n_steps' : [], 
                              't_min' : [], 
                              'state_tomo_fids_hardware' : [],
                              'mean_fid_hardware' : [],
                              'std_fid_hardware' : []}


for kind, jobs_ids in jobs_dict_1st_order_ids.items():
    
    print(f"\nType of circuit: {kind}")
    
    jobs = [jakarta.retrieve_job(job_id) for job_id in jobs_ids]
    
    status = [job._api_status for job in jobs]
    
    print(f"\nStatus of group of jobs (should be only 'COMPLETED'): {set(status)}\n")
    
    print(*[f"ID job {i+1}: {x._job_id}" for i, x in enumerate(jobs)], sep="\n")

    fids, fids_mean, fids_std = final_fidelities_retrieved(jobs)
    
    print()
    print("="*80)
    
    # ============================================
    
    order = int(kind.split("_")[1])
    steps = int(kind.split("_")[2])
    t_min = float(kind.split("_")[-1])
    
    results_hardware_1st_order["order"].append(order)
    results_hardware_1st_order["n_steps"].append(steps)
    results_hardware_1st_order["t_min"].append(t_min)
    results_hardware_1st_order["state_tomo_fids_hardware"].append(fids)
    results_hardware_1st_order["mean_fid_hardware"].append(fids_mean)
    results_hardware_1st_order["std_fid_hardware"].append(fids_std)
    
df_results_hardware_1st_order = pd.DataFrame(results_hardware_1st_order)


Type of circuit: order_1_4_steps_tmin_0.001

Status of group of jobs (should be only 'COMPLETED'): {'COMPLETED'}

ID job 1: 6250b2e9fdf3773fcb5700da
ID job 2: 6250b2f2fdf37739915700db
ID job 3: 6250b2fd182d02a6a04c38bd
ID job 4: 6250b306a5d4ee803377dbd6
ID job 5: 6250b30d4b5152b89f7c79ca
ID job 6: 6250b3144b515250a87c79cc
ID job 7: 6250b31bcfe45cfc8ee5b1cb
ID job 8: 6250b3225ab79b36f0f960d5

Final results for jobs above

State tomography fidelity = 0.3236 ± 0.0556



In [ ]:
results_hardware_2nd_order = {'order' : [], 
                              'n_steps' : [], 
                              't_min' : [], 
                              'state_tomo_fids_hardware' : [],
                              'mean_fid_hardware' : [],
                              'std_fid_hardware' : []}


for kind, jobs_ids in jobs_dict_2nd_order_ids.items():
    
    print(f"\nType of circuit: {kind}")
    
    jobs = [jakarta.retrieve_job(job_id) for job_id in jobs_ids]
    
    status = [job._api_status for job in jobs]
    
    print(f"\nStatus of group of jobs (should be only 'COMPLETED'): {set(status)}\n")
    
    print(*[f"ID job {i+1}: {x._job_id}" for i, x in enumerate(jobs)], sep="\n")

    fids, fids_mean, fids_std = final_fidelities_retrieved(jobs)
    
    print()
    print("="*80)
    
    # ============================================
    
    order = int(kind.split("_")[1])
    steps = int(kind.split("_")[2])
    t_min = float(kind.split("_")[-1])
    
    results_hardware_2nd_order["order"].append(order)
    results_hardware_2nd_order["n_steps"].append(steps)
    results_hardware_2nd_order["t_min"].append(t_min)
    results_hardware_2nd_order["state_tomo_fids_hardware"].append(fids)
    results_hardware_2nd_order["mean_fid_hardware"].append(fids_mean)
    results_hardware_2nd_order["std_fid_hardware"].append(fids_std)
    
df_results_hardware_2nd_order = pd.DataFrame(results_hardware_2nd_order)

## Merging hardware and simulation results

In [17]:
results_simulator_1st_order = pd.read_parquet('./results/results_opt_first_order_2022-04-08.parquet')
results_simulator_2nd_order = pd.read_parquet('results/results_opt_second_order_2022-04-06.parquet')

______________
______________
______________
______________
______________
______________

### By completion and date

(this was only a first test, it's not being used anymore)

In [11]:
completed_jobs = [x for x in jakarta.jobs(limit=1000) if x._api_status == 'COMPLETED']

dates_completed = [x._creation_date.date() for x in completed_jobs]

df_retrieved_jobs = pd.DataFrame({"job" : completed_jobs, "date" : dates_completed})

df_retrieved_jobs

,job,date
0,<qiskit.providers.ibmq.job.ibmqjob.IBMQJob obj...,2022-04-08
1,<qiskit.providers.ibmq.job.ibmqjob.IBMQJob obj...,2022-04-08
2,<qiskit.providers.ibmq.job.ibmqjob.IBMQJob obj...,2022-04-08
3,<qiskit.providers.ibmq.job.ibmqjob.IBMQJob obj...,2022-04-08
4,<qiskit.providers.ibmq.job.ibmqjob.IBMQJob obj...,2022-04-08
...,...,...
251,<qiskit.providers.ibmq.job.ibmqjob.IBMQJob obj...,2022-02-23
252,<qiskit.providers.ibmq.job.ibmqjob.IBMQJob obj...,2022-02-23
253,<qiskit.providers.ibmq.job.ibmqjob.IBMQJob obj...,2022-02-23
254,<qiskit.providers.ibmq.job.ibmqjob.IBMQJob obj...,2022-02-23


In [67]:
# for now, grouping by date
# the newer jobs were all sent in the same day.. so it'll probably be better
# to retrieve by the IDs. will do this later.
for date, jobs_in_date in df_retrieved_jobs.groupby("date"):
    
    print(f"\nJobs in {date}:\n")
    
    jobs = jobs_in_date["job"].values
    
    print(*[f"ID job {i+1}: {x._job_id}" for i, x in enumerate(jobs)], sep="\n")

    final_fidelities_retrieved(jobs)
    
    print()
    print("="*80)


Jobs in 2022-02-23:

ID job 1: 6215905857bc54332962bf55
ID job 2: 62159052bc41280455b6a897
ID job 3: 6215904c1710607e4badfcc6
ID job 4: 62159046bc4128549bb6a896
ID job 5: 6215904012d1b61ec4b81c04
ID job 6: 6215903aacb5ef2024d9850d
ID job 7: 62159034ffea130f2b1be8c1
ID job 8: 6215902d171060756aadfcc2

Final results for jobs above

State tomography fidelity = 0.5204 ± 0.0075


Jobs in 2022-04-04:

ID job 1: 624af66499622534288bb8a0
ID job 2: 624af657267dc73fccbd8a96
ID job 3: 624af64a133bfdf6165ac36c
ID job 4: 624af63db3213dfb2ac06576
ID job 5: 624af62f1a95e09af25cdfe4
ID job 6: 624af623be9f60092c771c2d
ID job 7: 624af611133bfd6b235ac36b
ID job 8: 624af604f9156f34f6cc05c8

Final results for jobs above

State tomography fidelity = 0.2466 ± 0.0137



___________

## Retrieve hardware execution with:

In [3]:
#try
order = 1
STEPS = 4
min_t = 0.01

In [4]:
# Completed jobs ID
jbsIDs= ['6249edcee978c730fdfb209e',
         '6249edd69962254bbb8bb657',
         '6249eddbf9156f2336cc034d',
         '6249eddf996225012a8bb658',
         '6249ede3133bfd15d95ac10a',
         '6249ede8be9f6036807719b6',
         '6249edec7130e1a7e35b2e54',
         '6249edef99622589258bb659']


In [5]:
jobsd = {}
for c,jbid in enumerate(jbsIDs):
    jobsd[c] = jakarta.retrieve_job(jbid)
    print(jbid,'LOADED!')

6249edcee978c730fdfb209e LOADED!
6249edd69962254bbb8bb657 LOADED!
6249eddbf9156f2336cc034d LOADED!
6249eddf996225012a8bb658 LOADED!
6249ede3133bfd15d95ac10a LOADED!
6249ede8be9f6036807719b6 LOADED!
6249edec7130e1a7e35b2e54 LOADED!
6249edef99622589258bb659 LOADED!


In [7]:
best_params = [0.01,  1.56,  0.01,  1.562]
st_qcs      = state_tomagraphy_circs(order, STEPS, uniform_times=False, steps_times=best_params)

In [10]:
# Compute tomography fidelities for each repetition
fids = []

for k in jobsd:
    fid = state_tomo(jobsd[k].result(), st_qcs)
    fids.append(fid)
    print(k,'DONE')

0 DONE
1 DONE
2 DONE
3 DONE
4 DONE
5 DONE
6 DONE
7 DONE


In [11]:
fids

[0.2071066452536354,
 0.20798413165329585,
 0.20981964638226672,
 0.20325432772496227,
 0.20882602038969406,
 0.20258631343542355,
 0.1972393408848579,
 0.20566002750692328]

In [12]:
print('State tomography fidelity = {:.4f} \u00B1 {:.4f}'.format(np.mean(fids), np.std(fids)))

State tomography fidelity = 0.2053 ± 0.0039
